In [2]:
import requests
import json
import os
from dotenv import load_dotenv
import pandas as pd


In [12]:
load_dotenv()

url = "https://financialmodelingprep.com/stable/search-name"
params = {
    "apikey": os.getenv("FMP_KEY"),
}

country_exchanges = {
    "Germany": ["XETRA", "DXE"],
    "Canada": ["TSX"],
    "United Kingdom": ["LSE"],
    "China": ["SHZ", "SHH"],
    "Spain": ["BME"],
    "Mexico": ["MEX"],
    "India": ["NSE"],
    "France": ["PAR"],
    "United States": ["NYSE", "NASDAQ", "AMEX", "OTC"],
    "Brazil": ["SAO"],
    "Japan": ["JPX"],
    "Hong Kong": ["HKSE"],
    "South Africa": ["JNB"],
    "Netherlands": ["AMS"],
    "Switzerland": ["SIX"],
    "Italy": ["MIL"],
    "Sweden": ["STO"],
    "Belize": [],
    "Denmark": ["CPH"],
    "Bermuda": [],
    "Kazakhstan": [],
    "Chile": ["SGO"],
    "Singapore": ["SGX"],
    "New Zealand": ["NZE"],
    "Belgium": ["BRU"],
    "Taiwan": ["TAI"],
    "South Korea": ["KSC"]
}


In [45]:
df = pd.read_csv("./data/FMP remapping - part2.csv")
df = df[df['Valid'] == "No Match"]
df.drop(['Valid', 'FMPTicker', 'FMP exchange','FMP name', 'Exchange country'], inplace=True, axis=1)
print(df.head())
print(len(df))

         db ticker market               name market Country
21        METSO_FI     DE              Metso        Germany
22         MONC_IT     DE            Moncler        Germany
24         MDBI_IT     DE         Mediobanca        Germany
113       LUNDB_SE     DE  Lundbergföretagen        Germany
114  LXI_GB_MERGED     DE           LXI REIT        Germany
511


In [ ]:
def clean(s):
    def fix_mojibake_to_ascii(s):
        s = str(s)
        try:
            # Step 1: Re-encode as bytes using the encoding that misinterpreted UTF-8
            bytes_data = s.encode('latin1')
            # Step 2: Decode properly using UTF-8
            fixed = bytes_data.decode('utf-8')
        except UnicodeEncodeError:
            # If encoding fails, fallback to original
            fixed = s
        except UnicodeDecodeError:
            # If decoding fails, fallback to original
            fixed = s

        # Step 3: Convert to ASCII (remove accents etc.)
        import unicodedata
        ascii_str = unicodedata.normalize('NFKD', fixed).encode('ascii', 'ignore').decode('ascii')
        return ascii_str
    s = fix_mojibake_to_ascii(s).lower().replace('.','').replace(',','').replace("'",'').replace('-',' ')
    
    redundant = ['ltd', 'inc', 'nv', 'sa', 'group', 'corporation', 
                 'corp', 'co', 'reit', 'limited', 'holding', 'holdings']
    words = [word for word in s.split() if word not in redundant]
    return ' '.join(words)
    


df['clean name'] = df['name'].apply(clean)
print(df.head())

In [48]:
df = pd.read_csv("./data/FMP remapping cleaned.csv")
print(df.head())

       db ticker market               name market Country         clean name
0       METSO_FI     DE              Metso        Germany              metso
1        MONC_IT     DE            Moncler        Germany            moncler
2        MDBI_IT     DE         Mediobanca        Germany         mediobanca
3       LUNDB_SE     DE  Lundbergföretagen        Germany  lundbergforetagen
4  LXI_GB_MERGED     DE           LXI REIT        Germany                lxi


In [49]:
symbols, names, tickers, fmp_names, exchanges, countries = ([] for i in range(6))

counter = 0

for company, name, ticker, country in zip(df['clean name'], df['name'], df['db ticker'], df['market Country']):
    print(company)
    
    params['query'] = company
    for exchange in country_exchanges[country]:
        params['exchange'] = exchange
        try:
            r = requests.get(url, params=params)
            if len(r.json()) == 0:
                raise Exception("No data")
            for res in r.json():
                names.append(name)
                tickers.append(ticker)
                countries.append(country)
                symbols.append(res['symbol'])
                fmp_names.append(res['name'])
                exchanges.append(res['exchange'])
                
        except Exception as e:
            counter += 1
            print(e, counter, country)
            names.append(company)
            tickers.append(ticker)
            countries.append(country)
            symbols.append('')
            fmp_names.append('')
            exchanges.append('')
#         
    
print(symbols)
        

metso
No data 1 Germany
No data 2 Germany
moncler
No data 3 Germany
mediobanca
No data 4 Germany
No data 5 Germany
lundbergforetagen
No data 6 Germany
No data 7 Germany
lxi
No data 8 Germany
No data 9 Germany
lonza
No data 10 Germany
No data 11 Germany
lindt
No data 12 Germany
No data 13 Germany
kerry
No data 14 Germany
No data 15 Germany
signify
No data 16 Germany
No data 17 Germany
kghm polska miedz
No data 18 Germany
No data 19 Germany
leonardo
No data 20 Germany
No data 21 Germany
koninklijke kpn
No data 22 Germany
No data 23 Germany
kuehn
No data 24 Germany
No data 25 Germany
kone
No data 26 Germany
kingspan
No data 27 Germany
No data 28 Germany
kongsberg gruppen
No data 29 Germany
No data 30 Germany
kbc
No data 31 Germany
No data 32 Germany
kesko
No data 33 Germany
lithium americas
haleon
nuvei
lightspeed commerce
just eat 
No data 34 Germany
No data 35 Germany
jde peet
No data 36 Germany
indutrade
No data 37 Germany
No data 38 Germany
intesa sanpaolo
No data 39 Germany
interpump

In [50]:
df2 = pd.DataFrame({
    'Ticker' : tickers,
    'Name' : names,
    'FMP name': fmp_names,
    'FMP symbol' : symbols,
    'Exchange' : exchanges,
    'Country' : countries
})
df2.to_csv(r'data/FMP remapping extracted 2.csv', index=False)

In [34]:
print(countries)

[]
